In [2]:
import os
import pandas as pd
import requests
import zipfile 
import re
from bs4 import BeautifulSoup

In [3]:
r = requests.get('https://download.cms.gov/nppes/NPI_Files.html')

In [4]:
r.text

'<!DOCTYPE html PUBLIC "-//W3C//DTD HTML 4.01 Transitional//EN" "http://www.w3c.org/TR/1999/REC-html401-19991224/loose.dtd">\n<!DOCTYPE html PUBLIC "-//W3C//DTD HTML 4.01 Transitional//EN" "http://www.w3.org/TR/html4/loose.dtd">\n<style type="text/css">\n\t.pStyle{\n\t\tpadding-left: 20px; margin: 0px;\n\t}\n    a:hover {\n\tcolor: rgb(255, 120, 0); text-decoration: underline;\n    }\n    .mainbox {\n\tmargin: 0px 0.5em; padding: 0px; border: 1px solid currentColor; font-family: "Lucida Grande", Verdana, Arial, Helvetica, sans-serif; font-size: 1em;\n    }\n    .mainbox h2 {\n\tpadding-left: 5px; font-size: 1em; margin:5px 0px 30px 0px;\n\t}\n    .header-background {\n\tbackground-image: url(\'images/Top_Banner1A.png\'); background-position: center; background-size: contain; height: 44px; align-items: center; display: flex; justify-content: flex-start; margin-top: 15px;\n\t}\n    .header-background h1 {\n\tfont-size: 2em; color: white; font-family: "Georgia"; font-weight: normal;\n    

In [5]:
soup = BeautifulSoup(r.text, 'html.parser')

# Find all zips
links = soup.find_all(href=True)
href_values = [os.path.basename(link['href']) for link in links if link['href'][-3:] == 'zip']

for href in href_values:
    print(href)

NPPES_Data_Dissemination_March_2025.zip
NPPES_Deactivated_NPI_Report_031025.zip
NPPES_Data_Dissemination_030325_030925_Weekly.zip
NPPES_Data_Dissemination_031025_031625_Weekly.zip
NPPES_Data_Dissemination_March_2025_V2.zip
NPPES_Deactivated_NPI_Report_031025_V2.zip
NPPES_Data_Dissemination_030325_030925_Weekly_V2.zip
NPPES_Data_Dissemination_031025_031625_Weekly_V2.zip


In [6]:
base_url = 'https://download.cms.gov/nppes/'

# now concat them to basepath and configurte output folder
outdir = '../data/'
if not os.path.isdir(outdir):
    os.mkdir(outdir)


In [25]:
# grab the last one as a test and save it to outidr. just loop through and save them.
# again not sure what struture you want and also at this point the files inside the zips could prob be condensed and reformmatted.
# The monthly file is huge so might need to do some delta reconcilation as you don't want to download the monthly file every day or week
r2 = requests.get(os.path.join(base_url, href_values[-2]))
filename = os.path.join(outdir, href_values[-2])
with open(filename, 'wb') as f:  
    f.write(r2.content)

In [26]:
# Extract the zip file
with zipfile.ZipFile(filename, 'r') as zip_ref:
    # List all files in the zip
    extracted_files = zip_ref.namelist()

    # Filter for CSV files
    csv_files = [file for file in extracted_files if file.endswith('.csv')]

    # Find the largest CSV file by comparing file sizes
    largest_csv_file = max(csv_files, key=lambda file: zip_ref.getinfo(file).file_size)

In [29]:
# Load the largest CSV into a pandas DataFrame
csv_path = os.path.join(outdir, largest_csv_file)
with zipfile.ZipFile(filename, 'r') as zip_ref:
    zip_ref.extract(largest_csv_file, outdir)  # Extract the largest CSV file

# Define dtype for the phone number column as string to avoid scientific notation
dtype_spec = {
    'Provider Business Mailing Address Telephone Number': str
}


df = pd.read_csv(csv_path)


C:\Users\alima\AppData\Local\Temp\ipykernel_15964\3049641098.py:12: DtypeWarning: Columns (17,24,32,68,75,76,77,78,79,80,81,82,83,84,85,86,87,88,89,90,91,92,93,94,95,96,97,98,99,100,101,102,103,104,105,106,123,127,131,133,134,135,137,138,139,141,142,143,145,146,147,149,150,151,153,154,155,157,158,159,161,162,163,165,166,167,169,170,171,173,174,175,177,178,181,185,189,193,195,197,201,205,207,209,213,217,221,223,225,227,229,233,235,237,241,312,316,317,318,319,320,321,322,323,324,325,326,327,328) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv(csv_path)


In [30]:
# Explicitly convert the phone number column to string (if necessary)
df['Provider Business Mailing Address Telephone Number'] = df['Provider Business Mailing Address Telephone Number'].astype(str).str.split('.').str[0]


# Extract the required columns
columns_needed = [
    'Provider First Name',
    'Provider Last Name (Legal Name)',
    'Provider Business Mailing Address State Name',
    'Provider Business Mailing Address Telephone Number'
]
# List of states to keep
valid_states = ['AL', 'AR', 'CO', 'DE', 'FL', 'GA', 'IA', 'IL', 'IN', 'KS', 'KY', 
                'LA', 'MD', 'MO', 'MI', 'MT', 'NC', 'NE', 'NV', 'OH', 'OK', 'SC', 
                'SD', 'TN', 'TX', 'UT', 'WI', 'WV', 'WY']



df_filtered = df[columns_needed]
df_filtered = df_filtered[df_filtered['Provider Business Mailing Address Telephone Number'].str.strip() != 'nan']
# Filter out rows where the state is not in the list
df_filtered = df_filtered[df_filtered['Provider Business Mailing Address State Name'].isin(valid_states)]
df_filtered = df_filtered.dropna(subset=['Provider Last Name (Legal Name)', 'Provider Business Mailing Address Telephone Number'])

# Display or save the filtered data
print(df_filtered.head())  # Display first few rows to verify



   Provider First Name Provider Last Name (Legal Name)  \
1           ALEXANDRIA                           BROWN   
2             BRITTANY                           LEWIS   
11             MELANIE                          LEVINE   
12             RADHIKA                      TUMMINELLO   
15            REGINALD                 NICHOLS-NKANSAH   

   Provider Business Mailing Address State Name  \
1                                            IL   
2                                            MI   
11                                           CO   
12                                           IL   
15                                           LA   

   Provider Business Mailing Address Telephone Number  
1                                          6186588331  
2                                          8557728847  
11                                         7342194175  
12                                         8663708206  
15                                         6304330178  


In [31]:
# Save the filtered data to a new CSV
df_filtered.to_csv(os.path.join(outdir, 'alexandra.csv'), index=False)

In [38]:
import random


business_types = [
    "chiropractor", "massage therapist", "acupuncturist", "physical therapist",
    "speech therapist", "lawyer", "accountant", "tax preparer", "financial advisor",
    "plumber", "electrician", "carpenter", "general contractor",
    "HVAC technician", "roofing contractor", "landscaper", "barber", "hair salon",
    "tattoo artist", "personal trainer", "nutritionist", "coffee shop owner",
    "bakery owner", "food truck owner", "boutique owner", "florist", "truck driver",
    "courier", "moving company owner", "freelance graphic designer", 
    "marketing consultant", "photographer", "videographer", "IT consultant",
    "web developer"
]

valid_states = {
    'AL': ['Birmingham', 'Montgomery', 'Huntsville'],
    'AR': ['Little Rock', 'Fort Smith', 'Fayetteville'],
    'CO': ['Denver', 'Colorado Springs', 'Aurora'],
    'DE': ['Wilmington', 'Dover', 'Newark'],
    'FL': ['Jacksonville', 'Miami', 'Tampa'],
    'GA': ['Atlanta', 'Augusta', 'Columbus'],
    'IA': ['Des Moines', 'Cedar Rapids', 'Davenport'],
    'IL': ['Chicago', 'Aurora', 'Naperville'],
    'IN': ['Indianapolis', 'Fort Wayne', 'Evansville'],
    'KS': ['Wichita', 'Overland Park', 'Kansas City'],
    'KY': ['Louisville', 'Lexington', 'Bowling Green'],
    'LA': ['New Orleans', 'Baton Rouge', 'Shreveport'],
    'MD': ['Baltimore', 'Columbia', 'Germantown'],
    'MO': ['Kansas City', 'St. Louis', 'Springfield'],
    'MI': ['Detroit', 'Grand Rapids', 'Warren'],
    'MT': ['Billings', 'Missoula', 'Great Falls'],
    'NC': ['Charlotte', 'Raleigh', 'Greensboro'],
    'NE': ['Omaha', 'Lincoln', 'Bellevue'],
    'NV': ['Las Vegas', 'Henderson', 'Reno'],
    'OH': ['Columbus', 'Cleveland', 'Cincinnati'],
    'OK': ['Oklahoma City', 'Tulsa', 'Norman'],
    'SC': ['Charleston', 'Columbia', 'North Charleston'],
    'SD': ['Sioux Falls', 'Rapid City', 'Aberdeen'],
    'TN': ['Nashville', 'Memphis', 'Knoxville'],
    'TX': ['Houston', 'San Antonio', 'Dallas'],
    'UT': ['Salt Lake City', 'West Valley City', 'Provo'],
    'WI': ['Milwaukee', 'Madison', 'Green Bay'],
    'WV': ['Charleston', 'Huntington', 'Morgantown'],
    'WY': ['Cheyenne', 'Casper', 'Laramie']
}

# Generate and print the queries
for state, cities in valid_states.items():
    query = f"{random.choice(business_types)};{random.choice(cities)}"
    print(query)


HVAC technician;Montgomery
plumber;Fort Smith
financial advisor;Denver
physical therapist;Dover
lawyer;Jacksonville
tax preparer;Columbus
accountant;Des Moines
roofing contractor;Chicago
HVAC technician;Fort Wayne
courier;Wichita
acupuncturist;Bowling Green
general contractor;New Orleans
plumber;Columbia
massage therapist;St. Louis
marketing consultant;Warren
massage therapist;Billings
marketing consultant;Raleigh
landscaper;Bellevue
roofing contractor;Henderson
bakery owner;Cincinnati
personal trainer;Oklahoma City
electrician;Columbia
food truck owner;Sioux Falls
florist;Nashville
chiropractor;Houston
nutritionist;West Valley City
web developer;Green Bay
financial advisor;Charleston
IT consultant;Casper
